# Playground Notebook
**Notebook to experiment with web scraping and text summarization.**

In [33]:
import requests
import urllib3
from bs4 import BeautifulSoup

from tqdm import tqdm

In [23]:
def fetchURL(url):
    response = requests.get(url)
    return response

fetchURL = lambda url : requests.get(url).data

fetchURL = lambda url: urllib3.PoolManager().request('GET', url).data

In [24]:
%%time
sampleUrl = 'https://www.rhsmith.umd.edu/directory'

html_content = fetchURL(sampleUrl)

soup = BeautifulSoup(html_content, 'lxml')

CPU times: user 24.3 ms, sys: 5.44 ms, total: 29.7 ms
Wall time: 225 ms


In [28]:
# soup

In [44]:
tagList = soup.findAll('a',attrs={'class':'fancy-link'})

In [49]:
len(tagList)
tag = tagList[0]

In [54]:
tag.attrs

{'class': ['fancy-link'], 'href': '/directory/suresh-acharya'}

In [60]:
# for tag in tqdm(tagList):
tag.get_text()

'Suresh Acharya'

In [64]:
tagList = soup.findAll('div',attrs={'class':"col-12 mb-4"})

In [65]:
len(tagList)

10

In [66]:
tagList[0]

<div class="col-12 mb-4">
<div class="row">
<div class="col-12 col-800-3 mb-3 mb-800-0">
<img alt="Suresh Acharya" class="w-100" src="/sites/default/files/people/headshots/acharya-suresh.jpg"/>
</div>
<div class="col-12 col-800-6">
<p class="h3 mb-1 person-name">
<a class="fancy-link" href="/directory/suresh-acharya">Suresh Acharya</a>
</p>
<ul><li>Professor of Practice</li><li>Academic Director, MS in Business Analytics</li></ul>
<div>Decision, Operations and Information Technologies</div>
<div>AI Faculty, Social Impact</div>
</div>
<div class="col-12 col-800-3">
<p class="profile-subheading">Contact</p>
<div>
<a href="tel:301-405-9678">301-405-9678</a>
</div>
<div>
<a href="mailto:suresh12@umd.edu">suresh12@umd.edu</a>
</div>
<div>4342 Van Munching Hall</div>
</div>
</div>
</div>